# Bible benchmark

First, some index definitions:

In [15]:
from typing import Iterable, List, Optional, Tuple, Union, NamedTuple, Dict
from abc import ABC, abstractmethod
from pydantic import BaseModel
import sys

logger = logging.getLogger('BibleCrawler')


class BibelReferenz(BaseModel):
    buch: str
    kapitel: int
    vers: Optional[int]
    bis_vers: Optional[int]


class BibelVers(BaseModel):
    number: int
    text: str
    references: List[BibelReferenz]


class BibelKapitel(BaseModel):
    """
    This is the representation of a "document" in our IR system.

    For more information, check out this resource:
    https://menora-bibel.jimdofree.com/aufbau-der-bibel/
    """
    testament: str
    buch: str
    kapitel: int
    verse: List[BibelVers]

    @property
    def text(self):
        return ' '.join([v.text for v in self.verse])


class TextProcessor(ABC):
    @abstractmethod
    def process_doc(self, src: Union[Iterable[str], str]) -> Iterable[str]:
        raise NotImplementedError("Diese Methode sollte Text, bzw. bereits tokenisierten Text weiter verarbeiten.")


class MemoryFullError(Exception):
    pass


class Posting(NamedTuple):
    document: int
    position: int


class IndexPointer(NamedTuple):
    offset: int
    num_postings: int


class InMemoryIndex(ABC):
    def __init__(self, memory_limit: int = 512):
        self.memory_limit = memory_limit

    def _get_size(self) -> int:
        """
        shamelessly copied from https://goshippo.com/blog/measure-real-size-any-python-object/
        """

        def get_size(obj, seen=None):
            """Recursively finds size of objects"""
            size = sys.getsizeof(obj)
            if seen is None:
                seen = set()
            obj_id = id(obj)
            if obj_id in seen:
                return 0
            # Important mark as seen *before* entering recursion to gracefully handle
            # self-referential objects
            seen.add(obj_id)
            if isinstance(obj, dict):
                size += sum([get_size(v, seen) for v in obj.values()])
                size += sum([get_size(k, seen) for k in obj.keys()])
            elif hasattr(obj, '__dict__'):
                size += get_size(obj.__dict__, seen)
            elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
                size += sum([get_size(i, seen) for i in obj])
            return size

        return get_size(self)

    def _is_full(self):
        return self._get_size() >= self.memory_limit

    def add(self, token: str, posting: Posting):
        self._add(token, posting)
        if self._is_full():
            raise MemoryFullError(f'InMemoryIndex contains more then {self.memory_limit} bytes!')

    @abstractmethod
    def __len__(self):
        """Diese Methode gibt die Anzahl der Einträge im Index zurück"""
        raise NotImplementedError

    @abstractmethod
    def clear(self):
        raise NotImplementedError("Diese Methode soll den in-memory Index komplett leeren.")

    @abstractmethod
    def _add(self, token: str, posting: Posting):
        """
        Diese Methode sollte das gegebene Posting in den temporären Index einfügen.
        Legen Sie sich dafür geeignete Instanzvariablen an um den Index zu speichern.
        Vermeiden Sie wenn möglich `dict()` und arbeiten Sie so platzsparend und
        effizient (CPU) wie möglich.
        """
        raise NotImplementedError("Diese Methode sollte das gegebene Posting in den temporären Index einfügen.")

    @abstractmethod
    def iterate_index(self) -> Iterable[Tuple[str, Iterable[Posting]]]:
        """
        Diese Methode sollte eine alphabetisch sortierte Liste via yield zurückgeben.
        Die Sortierung erfolgt aufsteigend (a-z) nach Token. Pro Token wird ein Tupel
        mit yield zurückgegeben, bestehend aus dem Token selbst und einer Liste an Postings.
        """
        raise NotImplementedError("Diese Methode sollte einen generator für speicherbare Elemente liefern.")


def get_file_size(file):
    file.seek(0, 2)
    return file.tell()


class SearchIndex(ABC):
    def __init__(self, documents: str, index_file: str,
                 temp_index: InMemoryIndex, pre_processing: List[TextProcessor]):
        self.documents_filename = documents
        self.index_filename = index_file
        self.pre_processing_pipeline = pre_processing
        self.temp_index = temp_index

    @abstractmethod
    def get_input_size(self) -> int:
        """
        Return size in bytes of the file `self.documents_filename`
        """
        raise NotImplementedError

    @abstractmethod
    def get_index_size(self) -> int:
        """
        Return size in byte of the index file.
        """
        raise NotImplementedError

    @abstractmethod
    def read_docs(self):
        """
        Dies ist die Haupt-Methode zur Konstruktion des Index.
        Sie iteriert alle Kapitel in der Eingabedatei und baut einen
        positional index. Schauen Sie zur weiten Erläuterung dazu bitte
        auf das Übungsblatt und das Video zum Übungsblatt.
        """
        raise NotImplementedError

    @abstractmethod
    def iterate_postings(self, num_postings, seek: int = None) -> Iterable[Posting]:
        """
        Diese Methode springt an eine Position in der Index Datei und liest ab dort
        `num_postings` viele Postings. Diese werden jeweils mit yield zurück gegeben.

        :param num_postings: Anzahl der Postings die ab seek gelesen werden können
        :param seek: wenn gesetzt, springt die Methode an die Position in der Index Datei
        """
        raise NotImplementedError

    @abstractmethod
    def get_index_lookup(self) -> Dict[str, Tuple[int, int]]:
        """
        Diese Methode gibt ein dict() zurück.
        Die Schlüssel (key) sind die im Index gespeicherten Token.
        Die Werte (value) sind Tupel:
            [0] = Offset in bytes (file.tell()) zum ersten Byte der Posting Liste des Tokens
            [1] = Anzahl der Postings für dieses Token

        Denken Sie daran, dass der Index nicht vollständig in den Hauptspeicher passt.
        Sie dürfen annehmen, dass der LookupIndex in dem Hauptspeicher passt.
        """
        raise NotImplementedError


In [16]:
from pydantic import AnyHttpUrl, BaseModel
from typing import Iterable, List, Optional, Tuple, Union
import re
import string


class SimpleTokenizer(TextProcessor):
    def process_doc(self, src: str) -> Iterable[str]:
        """
        Diese Funktion bekommt eine Zeichenkette und teilt diese in einzelne Token auf.
        Jedes erkannte Token wird einzeln via yield zurückgegeben.
        """
        # heavily inspired by
        # https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/parsing/preprocessing.py
        src = src.lower()
        src = src.replace('ä', 'oe')
        src = src.replace('ö', 'oe')
        src = src.replace('ü', 'oe')
        src = src.replace('ß', 'ss')
        src = re.sub(r'([%s])+' % re.escape(string.punctuation), ' ', src)
        src = re.sub(r'\s+', ' ', src)
        src = re.sub(r'[^a-zA-Z\- ]+', '', src)
        for token in src.split():
            yield token


class ShortTokenFilter(TextProcessor):
    def __init__(self, min_len=3):
        self.min_len = min_len

    def process_doc(self, src: Iterable[str]) -> Iterable[str]:
        """
        Diese Funktion erwartet eine Liste an Tokens.
        Es werden nur die Token einzeln via yield zurückgegeben, die eine minimale Länge überschreiten.
        """
        for token in src:
            if len(token) > self.min_len:
                yield token


class StopwordFilter(TextProcessor):
    def __init__(self, stopword_file):
        with open(stopword_file, 'r') as f:
            self.stopwords = set([stopword.strip() for stopword in f])

    def process_doc(self, src: Iterable[str]) -> Iterable[str]:
        """
        Diese Funktion erwartet eine Liste an Tokens.
        Es werden nur die Token einzeln via yield zurückgegeben, die nicht in einer
        Liste an Stopwörtern vorkommen.
        """
        for token in src:
            if token not in self.stopwords:
                yield token


class CustomStemmer(TextProcessor):

    def process_doc(self, src: Iterable[str]) -> Iterable[str]:
        """
        Hier können Sie einen beliebigen eigenen Stemmer oder Lemmatiser bauen.
        """
        for token in src:
            yield token


class CISTEMStemmer(TextProcessor):
    # heavily inspired by https://github.com/LeonieWeissweiler/CISTEM/blob/master/Cistem.py
    strip_ge = re.compile(r"^ge(.{4,})")
    repl_xx = re.compile(r"(.)\1")
    repl_xx_back = re.compile(r"(.)\*")
    strip_emr = re.compile(r"e[mr]$")
    strip_nd = re.compile(r"nd$")
    strip_t = re.compile(r"t$")
    strip_esn = re.compile(r"[esn]$")

    def __init__(self, case_insensitive=False):
        self.case_insensitive = case_insensitive

    def process_doc(self, src: Iterable[str]) -> Iterable[str]:
        for token in src:
            yield self.process_word(token)

    def process_word(self, wort: str) -> str:
        """
        Diese Funktion erwartet ein einzelnes Token und gibt dieses nach Anwendung des CISTEM Algorithmus zurück.
        Der Algorithmus wird in Abbildung 5 in der Veröffentlichung
            Developing a Stemmer for German Based on a Comparative Analysis of Publicly Available Stemmers
            Leonie Weißweiler, Alexander Fraser, 2017
        beschrieben.
        Link zum PDF: https://www.cis.uni-muenchen.de/~weissweiler/cistem/
        """
        if len(wort) == 0:
            return wort

        upper = wort[0].isupper()
        wort = wort.lower()

        wort = wort.replace("ü", "u")
        wort = wort.replace("ö", "o")
        wort = wort.replace("ä", "a")
        wort = wort.replace("ß", "ss")

        wort = self.strip_ge.sub(r"\1", wort)
        wort = wort.replace("sch", "$")
        wort = wort.replace("ei", "%")
        wort = wort.replace("ie", "&")
        wort = self.repl_xx.sub(r"\1*", wort)

        while len(wort) > 3:
            if len(wort) > 5:
                (wort, success) = self.strip_emr.subn("", wort)
                if success != 0:
                    continue

                (wort, success) = self.strip_nd.subn("", wort)
                if success != 0:
                    continue

            if not upper or self.case_insensitive:
                (wort, success) = self.strip_t.subn("", wort)
                if success != 0:
                    continue

            (wort, success) = self.strip_esn.subn("", wort)
            if success != 0:
                continue
            else:
                break

        wort = self.repl_xx_back.sub(r"\1\1", wort)
        wort = wort.replace("%", "ei")
        wort = wort.replace("&", "ie")
        wort = wort.replace("$", "sch")

        return wort


class PorterStemmer(TextProcessor):
    vokale = set('aeiouyäüö')
    konsonanten = set('bcdfghjklmnpqrstvwxzßUY')
    s_endung = set('bdfghklmnrt')
    st_endung = set('bdfghklmnt')
    stopliste = {'aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere',
                 'anderem', 'anderen', 'anderer', 'anderes', 'anderm', 'andern', 'anders', 'auch', 'auf', 'aus', 'bei',
                 'bin', 'bis', 'bist', 'da', 'damit', 'dann', 'der', 'den', 'des', 'dem', 'die', 'das', 'dass', 'daß',
                 'derselbe', 'derselben', 'denselben', 'desselben', 'demselben', 'dieselbe', 'dieselben', 'dasselbe',
                 'dazu', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'denn', 'derer', 'dessen', 'dich',
                 'dir', 'du', 'dies', 'diese', 'diesem', 'diesen', 'dieser', 'dieses', 'doch', 'dort', 'durch', 'ein',
                 'eine', 'einem', 'einen', 'einer', 'eines', 'einig', 'einige', 'einigem', 'einigen', 'einiger',
                 'einiges', 'einmal', 'er', 'ihn', 'ihm', 'es', 'etwas', 'euer', 'eure', 'eurem', 'euren', 'eurer',
                 'eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten', 'hier', 'hin',
                 'hinter', 'ich', 'mich', 'mir', 'ihr', 'ihre', 'ihrem', 'ihren', 'ihrer', 'ihres', 'euch', 'im', 'in',
                 'indem', 'ins', 'ist', 'jede', 'jedem', 'jeden', 'jeder', 'jedes', 'jene', 'jenem', 'jenen', 'jener',
                 'jenes', 'jetzt', 'kann', 'kein', 'keine', 'keinem', 'keinen', 'keiner', 'keines', 'können', 'könnte',
                 'machen', 'man', 'manche', 'manchem', 'manchen', 'mancher', 'manches', 'mein', 'meine', 'meinem',
                 'meinen', 'meiner', 'meines', 'mit', 'muss', 'musste', 'muß', 'mußte', 'nach', 'nicht', 'nichts',
                 'noch', 'nun', 'nur', 'ob', 'oder', 'ohne', 'sehr', 'sein', 'seine', 'seinem', 'seinen', 'seiner',
                 'seines', 'selbst', 'sich', 'sie', 'ihnen', 'sind', 'so', 'solche', 'solchem', 'solchen', 'solcher',
                 'solches', 'soll', 'sollte', 'sondern', 'sonst', 'über', 'um', 'und', 'uns', 'unse', 'unsem', 'unsen',
                 'unser', 'unses', 'unter', 'viel', 'vom', 'von', 'vor', 'während', 'war', 'waren', 'warst', 'was',
                 'weg', 'weil', 'weiter', 'welche', 'welchem', 'welchen', 'welcher', 'welches', 'wenn', 'werde',
                 'werden', 'wie', 'wieder', 'will', 'wir', 'wird', 'wirst', 'wo', 'wollem', 'wollte', 'würde', 'würden',
                 'zu', 'zum', 'zur', 'zwar', 'zwischen'}

    def __init__(self, use_stopwords=False):
        self.use_stopwords = use_stopwords

    def process_doc(self, src: Iterable[str]) -> Iterable[str]:
        for token in src:
            yield self.process_word(token)

    def process_word(self, wort: str) -> str:
        """
        Diese Funktion erwartet ein einzelnes Token und gibt dieses nach Anwendung des Porter Algorithmus zurück.
        Originale Veröffentlichung: https://www.cs.odu.edu/~jbollen/IR04/readings/readings5.pdf
        Adaption der Regeln für Deutsch: http://snowball.tartarus.org/algorithms/german/stemmer.html

        Keine Garantie, dass diese Implementierung perfekt korrekt ist...
        Dieser Code ist eine Übersetzung/Vereinfachung der Referenzimplementierung:
           http://snowball.tartarus.org/otherlangs/german_py.txt
        """
        if self.use_stopwords and (wort in self.stopliste):
            return wort

        wort = wort.replace('ß', 'ss')

        # Schützenswerte 'u' bzw. 'y' werden durch 'U' bzw. 'Y' ersetzt
        if 'u' in wort or 'y' in wort:
            for i, char in enumerate(wort):
                if i == 0 or i + 1 == len(wort):
                    continue
                if char == 'u' and wort[i - 1] in self.vokale and wort[i + 1] in self.vokale:
                    wort = wort[:i] + 'U' + wort[i + 1:]
                if char == 'y' and wort[i - 1] in self.vokale and wort[i + 1] in self.vokale:
                    wort = wort[:i] + 'Y' + wort[i + 1:]

        # r1, r2, p1 & p2 werden mit Werten belegt
        p1 = 0
        r1 = ''
        hat_vokal = False
        for i, char in enumerate(wort):
            if char in self.vokale:
                hat_vokal = True
            if char in self.konsonanten and hat_vokal:
                p1 = i + 1
                r1 = wort[p1:]
                break
        p2 = 0
        r2 = ''
        hat_vokal = False
        for i, char in enumerate(r1):
            if char in self.vokale:
                hat_vokal = True
            if char in self.konsonanten and hat_vokal:
                p2 = i + 1
                r2 = r1[p2:]
                break

        if 0 < p1 < 3:
            p1 = 3
            r1 = wort[p1:]
        if p1 == 0:
            return self.end_stemming(wort)

        # Schritt 1
        for suffix in ['e', 'em', 'en', 'ern', 'er', 'es']:
            suffix_len = len(suffix)
            if suffix == r1[-suffix_len:]:
                wort = wort[:-suffix_len]
                r1 = r1[:-suffix_len]
                r2 = r2[:-suffix_len]
                break
        else:  # only if loop didn't break
            if len(wort) >= 2 and len(r1) > 0 and \
                    r1[-1] == 's' and wort[-2] in self.s_endung:
                wort = wort[:-1]
                r1 = r1[:-1]
                r2 = r2[:-1]

        # Schritt 2
        for suffix in ['est', 'er', 'en']:
            suffix_len = len(suffix)
            if suffix == r1[-suffix_len:]:
                wort = wort[:-suffix_len]
                r1 = r1[:-suffix_len]
                r2 = r2[:-suffix_len]
                break
        else:  # only if loop didn't break
            if len(wort) > 5 and r1[-2:] == 'st' and wort[-3] in self.st_endung:
                wort = wort[:-2]
                r1 = r1[:-2]
                r2 = r2[:-2]

        # Schritt 3a)
        for suffix in ['end', 'ung']:
            suffix_len = len(suffix)
            if suffix == r2[-suffix_len:]:
                if 'ig' == r2[-suffix_len + 2:-suffix_len]:
                    if 'e' == wort[-suffix_len + 3]:
                        wort = wort[:-suffix_len]
                        r1 = r1[:-suffix_len]
                        r2 = r2[:-suffix_len]
                        break
                    else:
                        wort = wort[:-suffix_len + 2]
                        r1 = r1[:-suffix_len + 2]
                        r2 = r2[:-suffix_len + 2]
                        break
                else:
                    wort = wort[:-suffix_len]
                    r1 = r1[:-suffix_len]
                    r2 = r2[:-suffix_len]
                return self.end_stemming(wort)

        # Schritt 3b)
        for suffix in ['ig', 'ik', 'isch']:
            suffix_len = len(suffix)
            if suffix == r2[-suffix_len:]:
                if 'e' != wort[-suffix_len + 1]:
                    wort = wort[:-suffix_len]
                    r1 = r1[:-suffix_len]
                    r2 = r2[:-suffix_len]
                    break

        # Schritt 3c)
        for suffix in ['lich', 'keit']:
            suffix_len = len(suffix)
            if suffix == r2[-suffix_len:]:
                for suffix2 in ['er', 'en']:
                    suffix2_len = len(suffix2)
                    if suffix2 == r1[-(suffix_len + suffix2_len):-suffix_len]:
                        wort = wort[:-(suffix_len + suffix2_len)]
                        r1 = r1[:-(suffix_len + suffix2_len)]
                        r2 = r2[:-(suffix_len + suffix2_len)]
                        break
                else:  # only run if loop didn't break
                    wort = wort[:-suffix_len]
                    r1 = r1[:-suffix_len]
                    r2 = r2[:-suffix_len]
                    break

        # Schritt 3d)
        for suffix in ['keit']:
            suffix_len = len(suffix)
            if suffix == r2[-suffix_len:]:
                for suffix2 in ['lich', 'ig']:
                    suffix2_len = len(suffix2)
                    if suffix2 == r2[-(suffix_len + suffix2_len):-suffix_len]:
                        wort = wort[:-(suffix2_len + suffix_len)]
                        break
                else:  # only run if loop didn't break
                    wort = wort[:-suffix_len]
                    break
        return self.end_stemming(wort)

    def end_stemming(self, wort):
        wort = wort.replace('ä', 'a')
        wort = wort.replace('ö', 'o')
        wort = wort.replace('ü', 'u')
        wort = wort.replace('U', 'u')
        wort = wort.replace('Y', 'y')
        return wort


In [17]:
from typing import Iterable, Tuple, Union, Dict
from collections import defaultdict
import sys
import os
import time

class InMemoryBibelIndex(InMemoryIndex):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.storage = None
        self.init_storage()

    def __len__(self):
        return len(self.storage)

    def init_storage(self):
        self.storage = defaultdict(list)

    def _add(self, token: str, posting: Posting):
        self.storage[token].append(posting)

    def clear(self):
        old_size = self._get_size()

        del self.storage
        self.init_storage()

        print(f'Cleared in-memory index, size before: {old_size} and after: {self._get_size()}')

    def iterate_index(self) -> Iterable[Tuple[str, int, Iterable[Posting]]]:
        # print(self.storage)
        keys = sorted(self.storage.keys())
        for token in keys:
            yield token, len(self.storage[token]), iter(self.storage[token])


class BibelIndex(SearchIndex):
    DELIMITER = b'#'

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bibel_file = open(self.documents_filename, 'rb')
        self.index_file = open(self.index_filename, 'ab+')

    def _get_file_size(self, file) -> int:
        current_pos = file.tell()
        file.seek(0, 2)
        size = file.tell()
        file.seek(current_pos)
        return size

    def get_index_size(self) -> int:
        return self._get_file_size(self.index_file)

    def get_input_size(self) -> int:
        return self._get_file_size(self.bibel_file)

    def load_kapitel(self, seek: int = None) -> BibelKapitel:
        if seek is not None:
            self.bibel_file.seek(seek)
        line = self.bibel_file.readline().decode('utf-8')
        return BibelKapitel.parse_raw(line)

    def _iterate_specific_kapitel(self, seek_points: Iterable[int]) -> Iterable[BibelKapitel]:
        for seek_point in seek_points:
            yield self.load_kapitel(seek_point)

    def _iterate_kapitel(self) -> Tuple[int, int, BibelKapitel]:
        self.bibel_file.seek(0)
        i = -1
        while True:
            position = self.bibel_file.tell()
            line = self.bibel_file.readline()
            i += 1
            if len(line) < 1:
                break
            kapitel = BibelKapitel.parse_raw(line.decode('utf-8'))
            yield i, position, kapitel

    def _read_int(self, num_bytes: int, seek: int = None) -> int:
        """
        Read bytes from file and return integer.
        :param num_bytes: number of bytes to read
        :param seek: start byte to seek to, if empty, continue where the cursor currently is
        :return: integer converted from byte buffer
        """
        if seek is not None:
            self.index_file.seek(seek)

        buffer = self.index_file.read(num_bytes)
        return int.from_bytes(buffer, byteorder=sys.byteorder)

    def _int2bytes(self, number: int, num_bytes: int) -> bytes:
        """
        Diese Methode bekommt ein int `number` und konvertiert es
        in bytes der Länge `num_bytes`.
        """
        return number.to_bytes(length=num_bytes, byteorder=sys.byteorder, signed=False)

    def _read_token(self, seek: int = None) -> str:
        if seek is not None:
            self.index_file.seek(seek)

        buffer = b''
        while True:
            b = self.index_file.read(1)
            if b == b'':
                raise EOFError('Reached end of index file #EOF')
            if b == self.DELIMITER:
                return buffer.decode('utf-8')
            else:
                buffer += b

    def iterate_postings(self, num_postings, seek: int = None, conflict_safe=False) -> Iterable[Posting]:
        if seek is not None:
            self.index_file.seek(seek)
        position_lock = self.index_file.tell()
        for n in range(num_postings):
            if conflict_safe:
                self.index_file.seek(position_lock)
            doc = self._read_int(4)
            pos = self._read_int(2)
            position_lock = self.index_file.tell()
            yield Posting(document=doc, position=pos)

    def _iterate_index(self, include_offset=False) -> Union[Iterable[Tuple[str, int, Iterable[Posting]]],
                                                            Iterable[Tuple[str, int, int, Iterable[Posting]]]]:
        self.index_file.seek(0)
        try:
            while True:
                token = self._read_token()
                num_postings = self._read_int(3)
                if include_offset:
                    file_offset = self.index_file.tell()
                    yield token, file_offset, num_postings, self.iterate_postings(num_postings)
                else:
                    yield token, num_postings, self.iterate_postings(num_postings)
        except EOFError:
            pass

    def get_index_lookup(self) -> Dict[str, IndexPointer]:
        lookup = dict()
        for token, offset, num_postings, postings in self._iterate_index(include_offset=True):
            lookup[token] = IndexPointer(offset=offset, num_postings=num_postings)
            for _ in postings:  # have to read postings to get to next token
                pass
        return lookup

    def _posting2bytes(self, posting: Posting) -> bytes:
        ret = b''
        ret += self._int2bytes(posting.document, 4)
        ret += self._int2bytes(posting.position, 2)
        return ret

    def _merge_index(self):
        print('Merging in-memory index to disk.')
        start_time = time.time()

        temp_index_file = open(f'{self.index_filename}.tmp', 'wb+')

        stored_index = self._iterate_index()
        temp_index = self.temp_index.iterate_index()

        stored_token, num_stored_postings, stored_postings = next(stored_index, (None, 0, None))
        temp_token, num_temp_postings, temp_postings = next(temp_index, (None, 0, None))

        def start_token(token, num_postings):
            temp_index_file.write(token.encode('utf-8') + self.DELIMITER)
            temp_index_file.write(self._int2bytes(num_postings, 3))

        def write_posting(posting):
            temp_index_file.write(self._posting2bytes(posting))

        while True:
            if stored_token is None and temp_token is None:
                break

            # both tokens match, merge their postings
            if stored_token == temp_token:
                start_token(stored_token, num_stored_postings + num_temp_postings)
                # print('==', num_stored_postings, num_temp_postings, stored_token)

                stored_posting = next(stored_postings, None)
                temp_posting = next(temp_postings, None)

                while True:
                    if temp_posting is None and stored_posting is None:
                        break

                    if (temp_posting is not None) and (stored_posting is None or
                                                       temp_posting.document < stored_posting.document):
                        write_posting(temp_posting)
                        temp_posting = next(temp_postings, None)
                    elif (stored_posting is not None) and (temp_posting is None or
                                                           temp_posting.document >= stored_posting.document):
                        write_posting(stored_posting)
                        stored_posting = next(stored_postings, None)

                stored_token, num_stored_postings, stored_postings = next(stored_index, (None, 0, None))
                temp_token, num_temp_postings, temp_postings = next(temp_index, (None, 0, None))

            elif (stored_token is not None) and (temp_token is None or stored_token < temp_token):
                start_token(stored_token, num_stored_postings)
                # print('s', num_stored_postings, stored_token)
                for stored_posting in stored_postings:
                    write_posting(stored_posting)
                stored_token, num_stored_postings, stored_postings = next(stored_index, (None, 0, None))

            elif stored_token is None or stored_token >= temp_token:
                start_token(temp_token, num_temp_postings)
                # print('t', num_temp_postings, temp_token)
                for temp_posting in temp_postings:
                    write_posting(temp_posting)
                temp_token, num_temp_postings, temp_postings = next(temp_index, (None, 0, None))

        os.replace(temp_index_file.name, self.index_filename)
        self.index_file = temp_index_file
        used_time = time.time() - start_time
        print(f'Index merger done in {used_time:.3f}s')

    def read_docs(self):
        # clear (existing?) index file
        self.index_file.seek(0)
        self.index_file.truncate()

        bible_size = self.get_input_size()

        # iterate documents (here: bible chapters)
        for i, file_position, kapitel in self._iterate_kapitel():
            print(f'Processing "{kapitel.buch}" - {kapitel.kapitel}, '
                         f'file position: {file_position} ({(file_position / bible_size * 100):.2f}%)')
            start_time = time.time()
            # get text and apply pre-processing
            tokens = self.text_processing(kapitel.text)

            num_tokens = 0
            # iterate remaining normalised tokens in document
            for text_position, token in enumerate(tokens):
                num_tokens += 1
                try:
                    # add posting to in-memory index
                    self.temp_index.add(token, Posting(document=file_position, position=text_position))
                except MemoryFullError:
                    # when the index is full, flush-merge it to disk and clear it
                    self._merge_index()
                    self.temp_index.clear()
            used_time = time.time() - start_time
            print(f'Document had {num_tokens}, took {used_time:.3f}s @ {(num_tokens / used_time):.2f}tps | '
                         f'in-memory index contains {len(self.temp_index)} entries')

        # reading probably doesn't exactly finish with full index, so flush-merge rest to disk
        self._merge_index()

    def text_processing(self, raw_text) -> Iterable[str]:
        tokens = raw_text
        for processor in self.pre_processing_pipeline:
            tokens = processor.process_doc(tokens)
        yield from tokens


Next, the queries:

In [18]:
from abc import ABC, abstractmethod
from typing import Iterable, List, Tuple, Optional
from pydantic import BaseModel


class QueryKapitelResult(BaseModel):
    hits: List[Tuple[str, int]]
    rank_score: Optional[float] = 1.0
    kapitel: BibelKapitel

    def as_reference(self) -> str:
        return f'{self.kapitel.buch} {self.kapitel.kapitel}'

    def as_snippet(self) -> Tuple[str, str]:
        """
        Given a query result, return title and text snippet
        """
        window_size = 10
        centre_hit = self.hits[0]

        # We get the raw text here, however, the index works on tokenised, stemmed text
        # with stopwords and short tokens removed. So we start where the positing was in
        # the processed text and search forward till we (hopefully) find what we actually
        # want. This is not ideal, but a good enough heuristic.
        token_heuristic = self.kapitel.text.lower().split()
        for i in range(centre_hit[1], len(token_heuristic)):
            # if i < centre_hit[1]+2:
            if centre_hit[0].lower() in token_heuristic[i].lower():
                return self.as_reference(), ' '.join(token_heuristic[max(0, i - window_size):
                                                                     min(i + window_size, len(token_heuristic))])
        # Apparently the heuristic didn't work out, pretend the word is at the original position
        return self.as_reference(), ' '.join(token_heuristic[max(0, centre_hit[1] - window_size):
                                                             min(centre_hit[1] + window_size,
                                                                 len(token_heuristic))])


class QueryProcessor(ABC):

    @classmethod
    def to_reference_list(cls, results: Iterable[QueryKapitelResult]) -> Iterable[str]:
        for result in results:
            yield result.as_reference()

    @classmethod
    def to_text_result_list(cls, results: Iterable[QueryKapitelResult]) -> Iterable[str]:
        for result in results:
            yield result.as_snippet()

    @abstractmethod
    def query(self, query: str) -> Iterable[QueryKapitelResult]:
        """
        This method parses the query string and queries the index accordingly.
        It returns a tuple with the number of documents found and an iterable
        of documents (in this case bible chapters, that are yielded one by one).
        """
        raise NotImplementedError


In [19]:
from typing import Iterable, List, Optional, Tuple, NamedTuple, Union, Dict
import string
import sys
import os
import math
from enum import Enum
import time
from collections import defaultdict

class BinaryQueryProcessorMode(Enum):
    # order of query terms: ignore
    # ranking of documents: ignore
    # Korrespondiert mit Aufgabe 4b
    NON_POSITIONAL = 0

    # order of query terms: ignore
    # ranking of documents: based on term frequency
    # Korrespondiert mit Aufgabe 4c, d
    TF_RANKING = 1

    # order of query terms: ignore
    # ranking of documents: based on tf-idf average
    # Korrespondiert mit Aufgabe 4c, d
    TFIDF_RANKING = 2

    # order of query terms: yes
    # ranking of documents: based on tf-idf average
    # Korrespondiert mit aufgabe 4g
    POSITIONAL = 3


class IndexMetadataPointer(NamedTuple):
    # index_offset: ab dieser byte position in der Index-Datei beginnt die Posting Liste
    offset: int
    # num_postings: Anzahl der Postings, die ab der Position gespeichert sind
    # Anmerkung: äquivalent to TF (term frequency)
    num_postings: int
    # in so vielen verschiedenen dokumenten taucht das token auf (document frequency)
    num_documents: int


class BinaryQueryProcessor(QueryProcessor):
    def __init__(self, index: BibelIndex, mode: BinaryQueryProcessorMode, n_droppable: int = None, max_gap: int = 2,
                 simulate_sort=False):
        """
        In dieser Klasse implementieren Sie die nötigen Funktionen für Aufgabe 4b,c,d,g
        Die query() Methode ist schon vorbereitet, je nach `mode` implementieren Sie die entsprechenden
        weiteren Methoden um eine Index-basierte Volltextsuche zu ermöglichen. Bedenken Sie wie
        auch beim letzten Mal, dass alle Operationen möglichst keine Listen im Speicher nutzen,
        da die in einem realen IR System zu groß für den Arbeitsspeicher wären.

        :param index: geladener BibelIndex
        :param mode: Modus
        :param simulate_sort: this will break our memory-safe scheme and sort the result
        :param max_gap: OPTIONAL: für 4g, wenn Lücken im Text zwischen Query Termen erlaubt sein sollen
        :param n_droppable: OPTIONAL: können Sie nutzen um zuzulassen, dass n Token des Queries nicht vorkommen müssen
                           (Musterlösung ignoriert das)
        """
        self.mode = mode
        self.index = index
        self.simulate_sort = simulate_sort
        self.max_gap = max_gap
        self.n_droppable = n_droppable

        print('Loading lookup table...')
        self.lookup = index.get_index_lookup()

        self.num_docs = 0
        if mode == BinaryQueryProcessorMode.POSITIONAL or mode == BinaryQueryProcessorMode.TFIDF_RANKING:
            print('Initialising additional meta-data...')
            self.lookup_meta = self._init_metadata()

        print(f'Loaded index with {len(self.lookup)} words in vocab and {self.num_docs} documents '
                     f'with mode={mode}.')

    def beispiel_index_nutzung(self):
        # Ein Token zu dem wir die Posting-Liste haben wollen
        suchterm = 'klug'

        # offset: ab dieser byte position in der Index-Datei beginnt die Posting Liste
        # num_postings: Anzahl der Postings, die ab der Position gespeichert sind
        index_pointer = self.lookup[suchterm]

        # Ein Iterator an postings. Also keine Liste und es wäre (theoretisch) zu groß mit
        # list(postings) den Iterator als Liste im Speicher zu persistieren.
        postings = self.index.iterate_postings(index_pointer.num_postings, seek=index_pointer.offset)

        last_doc = None
        hits = []
        score = 0.0
        for posting in postings:
            # Ein Posting besteht aus Dokument und Position im Dokument
            print(posting)

            # So kann man ein Kapitel laden
            kapitel = self.index.load_kapitel(posting.document)

            # Zu beachten: posting.position verweist nicht unbedingt
            #              auf das richtige Token in kapitel.text
            # Der richtige Verweis wäre:
            #      self.index.text_processing(kapitel.text)[posting.position]
            # Benötigt man aber nicht für die Aufgaben.
            print(f'Das Token "{suchterm}" kommt in Kapitel '
                  f'"{kapitel.buch} {kapitel.kapitel}" Stelle {posting.position} vor.')

            score += 1
            hits.append((suchterm, posting.position))

            if last_doc is None or last_doc != posting.document:
                yield QueryKapitelResult(hits=hits, kapitel=kapitel, rank_score=score)
                last_doc = posting.document
                hits = []

        # Hinweis: das letzte Kapitel würde hier noch zurückgegeben werden...

    def beispiel_postings_iterieren(self):
        term1 = 'brot'
        term2 = 'tal'

        iteratoren = {}

        index_pointer = self.lookup[term1]
        iteratoren[term1] = self.index.iterate_postings(index_pointer.num_postings, seek=index_pointer.offset)
        index_pointer = self.lookup[term2]
        iteratoren[term2] = self.index.iterate_postings(index_pointer.num_postings, seek=index_pointer.offset)

        print(next(iteratoren[term1]))
        print(next(iteratoren[term1]))
        print(next(iteratoren[term2]))
        print(next(iteratoren[term1]))
        print(next(iteratoren[term2]))
        print(next(iteratoren[term2]))
        print(next(iteratoren[term1]))
        print(next(iteratoren[term2]))
        # irgendwas stimmt doch hier nicht...

        # iterate_postings iteriert auf der index Datei. Angenommen zwei Iteratoren werden abwechselnd
        # iteriert, dann bewegt sich der pointer weiter, aber das Ergebnis macht keinen Sinn.
        # Daher gibt es den conflict_safe modus, der merkt sich bei jedem Schritt für jeden der
        # Iteratoren die Position in der Datei und springt falls nötig hin und her.
        index_pointer = self.lookup[term1]
        iteratoren[term1] = self.index.iterate_postings(index_pointer.num_postings,
                                                        seek=index_pointer.offset, conflict_safe=True)
        index_pointer = self.lookup[term2]
        iteratoren[term2] = self.index.iterate_postings(index_pointer.num_postings,
                                                        seek=index_pointer.offset, conflict_safe=True)

        print(next(iteratoren[term1]))
        print(next(iteratoren[term1]))
        print(next(iteratoren[term2]))
        print(next(iteratoren[term1]))
        print(next(iteratoren[term2]))
        print(next(iteratoren[term2]))
        print(next(iteratoren[term1]))
        print(next(iteratoren[term2]))

    def _init_metadata(self) -> Dict[str, IndexMetadataPointer]:
        """
        Diese Methode iteriert den vorhandenen self.lookup und ersetzt
        alle Einträge (IndexPointer) mit IndexMetadataPointer um so
        angereicherte Informationen zu haben.
        Sie dürfen zur Vereinfachung davon ausgehen, dass die Liste der Document IDs
        in den Hauptspeicher passt.
        """
        metadata_lookup = defaultdict(lambda: IndexMetadataPointer(offset=0, num_postings=0, num_documents=0))
        all_docs = set()
        for term in self.lookup.keys():
            offset, num_postings = self.lookup[term]
            docs_with_t = set(map(lambda posting: posting.document, self.index.iterate_postings(num_postings, seek=offset)))
            metadata_lookup[term] = IndexMetadataPointer(offset=offset, num_postings=num_postings, num_documents=len(docs_with_t))
            all_docs.update(docs_with_t)
        # print(f'all docs are {all_docs}')
        self.num_docs = len(all_docs)
        return metadata_lookup

    def _tf_idf(self, token: str, tf: int):
        """
        Bekommt das Token und die Term Frequency des Tokens im aktuellen Dokument.
        Gibt den TF-IDF Score zurück.
        Hinweise hier: https://en.wikipedia.org/wiki/Tf%E2%80%93idf
        Sie können eine beliebige Variation nutzen.
        Tests nutzen: log(1 + tf(t, d)) * log(N / num docs mit t)

        Nötige Metadaten sollten vorab mit self._init_metadata vorbereitet werden.
        """
        metadata = self.lookup_meta[token]
        if metadata.num_documents == 0:
            # Tokens that don't appear at all get the lowest score, 0.
            return 0
        return math.log(1 + tf) * math.log(self.num_docs / metadata.num_documents)

    def _query_non_positional(self, query: List[str]) -> Iterable[QueryKapitelResult]:
        """
        Gegeben ein bereits geparster Query.
        Nutzen Sie den Index um effizient die Anfrage auszuführen und passende Dokumente zu finden.
        Versuchen Sie zu keinem Zeitpunkt eine Ergebnisliste oder größere Iteratoren im Speicher zu halten.
        In einem web-scale index wäre das auch nicht möglich.

        Tipp: Sie können den Algorithmus von merkeIndexPartitions aus dem Lehrbuch
              clever anpassen und mit ein Mail über die relevanten Postings iterieren
              eine Ergebnisliste erzeugen. Sie müssen nur dort, wo addPostings passiert
              noch prüfen, ob wirklich alle Terme aus der Anfrage dabei sind.
        http://www.ir.uwaterloo.ca/book/04-static-inverted-indices.pdf#page=26
        """
        # Vorgehen: Es gibt mehrere Iteratoren – für jeden Suchbegriff einen. Wir suchen nur
        # Kapitel, in denen alle Terme vorkommen (konjunktiv). Der maximal weite Iterator gibt den
        # nächsten Kapitelkandidaten an (weil es muss ja möglich sein, von jedem Iterator noch zu
        # dem Kapitel zu kommen). Dann advancen wir alle Iteratoren so lange, bis sie entweder auch
        # bei dem Kapitel sind (der jeweilige Suchbegriff existert also auch im Kapitel – ab da
        # können wir die Postings speichern) oder sie haben kein Posting in diesem Kapitel
        # (der jeweilige Suchbegriff existiert in dem Kapitel nicht und wir können das Kapitel als
        # Kandidaten verwerfen).

        # Dictionary von Suchtermen zu Iteratoren.
        iteratoren = {}
        for term in query:
            try:
                index_pointer = self.lookup[term]
                iteratoren[term] = self.index.iterate_postings(index_pointer.num_postings,
                                                        seek=index_pointer.offset, conflict_safe=True)
            except KeyError:
                # Einer der Suchbegriffe existiert nicht mal im Index. Das ist natürlich für den
                # User doof, macht es aber einfach für uns.
                return

        # Wir holen die aktuellen Postings von allen Iteratoren.
        # NOTE: Das kann eleganter ohne Extra-Variable durch peek() gelöst werden, aber dazu müsste
        # man aber entweder selbst ein peekable implementieren, oder ein weiteres externes package
        # nutzen, z.B. more-itertools.
        iteratoren_postings = {}
        for term in query:
            # Dies wirft keine StopIteration, weil Suchbegriffe, die im Index vorkommen, mindestens
            # einen Posting-Eintrag haben.
            iteratoren_postings[term] = next(iteratoren[term])

        abort = False
        while not abort:
            # Das nächste Kapitel muss das maximale von allen Iteratoren sein. Für die anderen
            # existiert mindestens ein Iterator, der bereits weiter ist – also ein Suchbegriff, den
            # es in dem Kapitel nicht gibt.
            kapitel_candidate = max(map(lambda p: p.document, iteratoren_postings.values()))
            found_terms = set()
            hits = []
            for term in query:
                # Zunächst Postings überspringen, bis wir beim gesuchten Kapitel sind.
                while iteratoren_postings[term].document < kapitel_candidate:
                    # Wir haben den Kandidaten noch nicht erreicht.
                    try:
                        iteratoren_postings[term] = next(iteratoren[term])
                    except StopIteration:
                        # Wir haben alle Vorkommnisse dieses Suchbegriffs abgefrühstückt. Das heißt,
                        # es ergibt keinen Sinn, noch weiter nach Kapiteln zu suchen.
                        abort = True
                        break
                if abort:
                    break

                if iteratoren_postings[term].document > kapitel_candidate:
                    # Der Kapitelkandidat wurde übersprungen. Das heißt, den Suchbegriff gibt es gar
                    # nicht in dem Kapitel. Schade. Dann suchen wir uns mal den nächsten Kandidaten
                    # raus!
                    continue

                # Der Iterator ist jetzt bei dem gesuchten Kapitel – es gibt tatsächlich Postings
                # für den Suchterm in diesem Kapitel. Wir suchen jetzt noch weiter, bis wir alle
                # Auftreten des Suchbegriffs gefunden haben.
                found_terms.add(term)
                while iteratoren_postings[term].document == kapitel_candidate:
                    hits.append([term, iteratoren_postings[term].position])
                    try:
                        iteratoren_postings[term] = next(iteratoren[term])
                    except StopIteration:
                        # Dieser Iterator ist fertig. Wir machen erstmal weiter und schauen ob die
                        # anderen Suchbegriffe auch im Kapitel sind. Nach diesem Durchlauf hören wir
                        # dann aber auf.
                        abort = True
                        break

            if len(found_terms) == len(set(query)):
                # print(f'Found {query} in {kapitel_candidate}. hits={hits} (found={found_terms} query={query})')
                yield QueryKapitelResult(hits=hits, kapitel=self.index.load_kapitel(kapitel_candidate))

    def _query_tf_ranked(self, query: List[str]) -> Iterable[QueryKapitelResult]:
        """
        Gleiches wie oben, nur, dass in der QueryKapitelResult auch der rank_score mit ausgefüllt wird.
        Bei dieser Funktion ist der Score die Summe der Term Frequency im Dokument
        (überlappend mit query, einfach, ohne Normalisierung)
        """
        # Vorgehen: s.o.

        # Dictionary von Suchtermen zu Iteratoren.
        iteratoren = {}
        for term in query:
            try:
                index_pointer = self.lookup[term]
                iteratoren[term] = self.index.iterate_postings(index_pointer.num_postings,
                                                        seek=index_pointer.offset, conflict_safe=True)
            except KeyError:
                # Einer der Suchbegriffe existiert nicht mal im Index. Das ist natürlich für den
                # User doof, macht es aber einfach für uns.
                return

        # Wir holen die aktuellen Postings von allen Iteratoren.
        # NOTE: Das kann eleganter ohne Extra-Variable durch peek() gelöst werden, aber dazu müsste
        # man aber entweder selbst ein peekable implementieren, oder ein weiteres externes package
        # nutzen, z.B. more-itertools.
        iteratoren_postings = {}
        for term in query:
            # Dies wirft keine StopIteration, weil Suchbegriffe, die im Index vorkommen, mindestens
            # einen Posting-Eintrag haben.
            iteratoren_postings[term] = next(iteratoren[term])

        abort = False
        while not abort:
            # Das nächste Kapitel muss das maximale von allen Iteratoren sein. Für die anderen
            # existiert mindestens ein Iterator, der bereits weiter ist – also ein Suchbegriff, den
            # es in dem Kapitel nicht gibt.
            kapitel_candidate = max(map(lambda p: p.document, iteratoren_postings.values()))
            found_terms = set()
            hits = []
            for term in query:
                # Zunächst Postings überspringen, bis wir beim gesuchten Kapitel sind.
                while iteratoren_postings[term].document < kapitel_candidate:
                    # Wir haben den Kandidaten noch nicht erreicht.
                    try:
                        iteratoren_postings[term] = next(iteratoren[term])
                    except StopIteration:
                        # Wir haben alle Vorkommnisse dieses Suchbegriffs abgefrühstückt. Das heißt,
                        # es ergibt keinen Sinn, noch weiter nach Kapiteln zu suchen.
                        abort = True
                        break
                if abort:
                    break

                if iteratoren_postings[term].document > kapitel_candidate:
                    # Der Kapitelkandidat wurde übersprungen. Das heißt, den Suchbegriff gibt es gar
                    # nicht in dem Kapitel. Schade. Dann suchen wir uns mal den nächsten Kandidaten
                    # raus!
                    continue

                # Der Iterator ist jetzt bei dem gesuchten Kapitel – es gibt tatsächlich Postings
                # für den Suchterm in diesem Kapitel. Wir suchen jetzt noch weiter, bis wir alle
                # Auftreten des Suchbegriffs gefunden haben.
                found_terms.add(term)
                while iteratoren_postings[term].document == kapitel_candidate:
                    hits.append([term, iteratoren_postings[term].position])
                    try:
                        iteratoren_postings[term] = next(iteratoren[term])
                    except StopIteration:
                        # Dieser Iterator ist fertig. Wir machen erstmal weiter und schauen ob die
                        # anderen Suchbegriffe auch im Kapitel sind. Nach diesem Durchlauf hören wir
                        # dann aber auf.
                        abort = True
                        break

            if len(found_terms) == len(set(query)):
                yield QueryKapitelResult(hits=hits, rank_score=len(hits), kapitel=self.index.load_kapitel(kapitel_candidate))

    def _query_tfidf_ranked(self, query: List[str]) -> Iterable[QueryKapitelResult]:
        """
        Gleiches wie oben, nur, dass in der QueryKapitelResult auch der rank_score mit ausgefüllt wird.
        Bei dieser Funktion ist der Score die Summe der TF-IDF scores
        """
        # Vorgehen: s.o.

        # Dictionary von Suchtermen zu Iteratoren.
        iteratoren = {}
        for term in query:
            try:
                index_pointer = self.lookup[term]
                iteratoren[term] = self.index.iterate_postings(index_pointer.num_postings,
                                                        seek=index_pointer.offset, conflict_safe=True)
            except KeyError:
                # Einer der Suchbegriffe existiert nicht mal im Index. Das ist natürlich für den
                # User doof, macht es aber einfach für uns.
                return

        # Wir holen die aktuellen Postings von allen Iteratoren.
        # NOTE: Das kann eleganter ohne Extra-Variable durch peek() gelöst werden, aber dazu müsste
        # man aber entweder selbst ein peekable implementieren, oder ein weiteres externes package
        # nutzen, z.B. more-itertools.
        iteratoren_postings = {}
        for term in query:
            # Dies wirft keine StopIteration, weil Suchbegriffe, die im Index vorkommen, mindestens
            # einen Posting-Eintrag haben.
            iteratoren_postings[term] = next(iteratoren[term])

        abort = False
        while not abort:
            # Das nächste Kapitel muss das maximale von allen Iteratoren sein. Für die anderen
            # existiert mindestens ein Iterator, der bereits weiter ist – also ein Suchbegriff, den
            # es in dem Kapitel nicht gibt.
            kapitel_candidate = max(map(lambda p: p.document, iteratoren_postings.values()))
            found_terms = set()
            hits = []
            for term in query:
                # Zunächst Postings überspringen, bis wir beim gesuchten Kapitel sind.
                while iteratoren_postings[term].document < kapitel_candidate:
                    # Wir haben den Kandidaten noch nicht erreicht.
                    try:
                        iteratoren_postings[term] = next(iteratoren[term])
                    except StopIteration:
                        # Wir haben alle Vorkommnisse dieses Suchbegriffs abgefrühstückt. Das heißt,
                        # es ergibt keinen Sinn, noch weiter nach Kapiteln zu suchen.
                        abort = True
                        break
                if abort:
                    break

                if iteratoren_postings[term].document > kapitel_candidate:
                    # Der Kapitelkandidat wurde übersprungen. Das heißt, den Suchbegriff gibt es gar
                    # nicht in dem Kapitel. Schade. Dann suchen wir uns mal den nächsten Kandidaten
                    # raus!
                    continue

                # Der Iterator ist jetzt bei dem gesuchten Kapitel – es gibt tatsächlich Postings
                # für den Suchterm in diesem Kapitel. Wir suchen jetzt noch weiter, bis wir alle
                # Auftreten des Suchbegriffs gefunden haben.
                found_terms.add(term)
                while iteratoren_postings[term].document == kapitel_candidate:
                    hits.append([term, iteratoren_postings[term].position])
                    try:
                        iteratoren_postings[term] = next(iteratoren[term])
                    except StopIteration:
                        # Dieser Iterator ist fertig. Wir machen erstmal weiter und schauen ob die
                        # anderen Suchbegriffe auch im Kapitel sind. Nach diesem Durchlauf hören wir
                        # dann aber auf.
                        abort = True
                        break

            if len(found_terms) == len(set(query)):
                score = 0
                for term in query:
                    num_hits = sum(map(lambda hit: 1 if hit[0] == term else 0, hits))
                    score += self._tf_idf(term, num_hits)
                yield QueryKapitelResult(hits=hits, rank_score=score, kapitel=self.index.load_kapitel(kapitel_candidate))

    def _query_positional(self, query: List[str]) -> Iterable[QueryKapitelResult]:
        """
        Gleiches wie tf-idf query mit der Erweiterung, dass die Wortreihenfolge im Query eine Rolle spielt.
        Sie können die entsprechenden Klassenvariablen nutzen um Abstände zwischen den Wörtern zu erlauben oder
        sogar ganze Wörter ausfallen zu lassen. Wortabstand ist zu empfehlen, da ggf. Stoppwörter fehlen.
        """
        raise NotImplementedError

    def query(self, query: str) -> Iterable[QueryKapitelResult]:
        query_terms = list(self.index.text_processing(query))
        # print(f'Query "{query}" transformed to {query_terms}')

        # oops, nothing left from the query
        if len(query_terms) < 1:
            return

        if self.mode == BinaryQueryProcessorMode.NON_POSITIONAL:
            yield from self._query_non_positional(query_terms)
        elif self.mode == BinaryQueryProcessorMode.TF_RANKING:
            results = self._query_tf_ranked(query_terms)
            if self.simulate_sort:
                # This is not smart at all!
                # This goes against all our efforts to keep everything out of memory.
                # Assume, this never happened and we have a black box that can sort our stuff out-of-memory!
                for r in sorted(list(results),
                                key=lambda r: (r.rank_score, r.kapitel.buch, r.kapitel.kapitel), reverse=True):
                    yield r
            else:
                yield from results
        elif self.mode == BinaryQueryProcessorMode.TFIDF_RANKING:
            results = self._query_tfidf_ranked(query_terms)
            if self.simulate_sort:
                for r in sorted(list(results),
                                key=lambda r: (r.rank_score, r.kapitel.buch, r.kapitel.kapitel), reverse=True):
                    yield r
            else:
                yield from results
        elif self.mode == BinaryQueryProcessorMode.POSITIONAL:
            results = self._query_positional(query_terms)
            if self.simulate_sort:
                for r in sorted(list(results),
                                key=lambda r: (r.rank_score, r.kapitel.buch, r.kapitel.kapitel), reverse=True):
                    yield r
            else:
                yield from results
        else:
            logger.fatal('Nope.')


In [20]:
from typing import Iterable, List, Tuple
from collections import defaultdict

class FullScanQueryProcessor(QueryProcessor):
    def __init__(self, documents_filename: str, pipeline: List[TextProcessor]):
        self.documents_filename = documents_filename
        self.pipeline = pipeline

    def _kapitel_iter(self) -> Iterable[BibelKapitel]:
        with open(self.documents_filename, 'r') as f:
            for line in f:
                yield BibelKapitel.parse_raw(line)

    def _tokenise(self, text: str) -> Iterable[str]:
        for processor in self.pipeline:
            text = processor.process_doc(text)
        yield from text

    def query(self, query: str) -> Iterable[QueryKapitelResult]:
        """
        Bekommt einen Query String und gibt die BibelKapitel zurück, in denen alle
        Token gefunden wurden. Das Tuple[str, int] sind die einzelnen Treffer im Dokument,
        bestehend aus dem entsprechenden Query-Token und der Stelle im Dokument (BibelKapitel.text).
        Nutzen Sie keine in Python eingebaute string Suche (etwa 'needle' in 'heystack with neelde').

        In Kapitel 10 aus "Information Retrieval: Data Structures & Algorithms"
        gibt es einige Algorithmen zur String Suche ohne Index. Es genügt vollkommen, wenn
        Sie eine einfache Brute-force Suche implementieren:
        for each kapitel:
          for each token in kapitel:
            for each query_token:
              if token == query_token: merken
          liste gemerkter token == liste query_token: kapitel passt auf query.
        https://cdn.preterhuman.net/texts/math/Data_Structure_And_Algorithms/Information%20Retrieval%20Data%20Structures%20&%20Algorithms%20-%20William%20B.%20Frakes.pdf#page=293&zoom=100,0,0
        """
        query_tokens = set(self._tokenise(query))
        for kapitel in self._kapitel_iter():
            hits: List[Tuple[str, int]] = []
            found_query_tokens = set()
            for pos, token in enumerate(self._tokenise(kapitel.text)):
                for query_token in query_tokens:
                    if token == query_token:
                        hits.append((token, pos))
                        found_query_tokens.add(token)
            if  found_query_tokens == query_tokens:
                yield QueryKapitelResult(hits=hits, kapitel=kapitel)


Then, for the main part:

In [21]:
import time

bibel_stemmer = PorterStemmer()

PRE_PROCESSING = [
    SimpleTokenizer(),
    ShortTokenFilter(min_len=2),
    StopwordFilter(stopword_file='bible-stopwords.txt'),
    bibel_stemmer
]

temporary_index = InMemoryBibelIndex(memory_limit=100 * 1024)
bibel_index = BibelIndex(documents='bible.json',
                         pre_processing=PRE_PROCESSING,
                         temp_index=temporary_index,
                         index_file='inverted_index.bin')

print('Initialised BibelIndex, reading chapters...')
bibel_index.read_docs()

lookup = bibel_index.get_index_lookup()
print('== BibelIndex Stats ==')

most_popular = sorted(lookup.items(), key=lambda l: l[1][1], reverse=True)

print('~~ most popular tokens ~~')
for token, (offset, tf) in most_popular[:20]:
    print(f'"{token}" appeared {tf} times, posting starts at '
                f'{(offset / bibel_index.get_index_size() * 100):.2f}% of the file')

print('~~ least popular tokens ~~')
for token, (offset, tf) in reversed(most_popular[-20:]):
    print(f'"{token}" appeared {tf} times, posting starts at '
                f'{(offset / bibel_index.get_index_size() * 100):.2f}% of the file')

print(f'Number of distinct tokens: {len(lookup)}')
print(f'Index Size: {(bibel_index.get_index_size() / 1024):.2f}KB')
print(f'Bible Size: {(bibel_index.get_input_size() / 1024):.2f}KB')

Initialised BibelIndex, reading chapters...
Processing "Genesis/1. Mose" - 2, file position: 0 (0.00%)
Document had 240, took 0.063s @ 3786.38tps | in-memory index contains 124 entries
Processing "Exodus/2. Mose" - 1, file position: 4654 (0.07%)
Document had 170, took 0.108s @ 1567.39tps | in-memory index contains 221 entries
Processing "Levitikus/3. Mose" - 1, file position: 8476 (0.12%)
Document had 182, took 0.147s @ 1241.03tps | in-memory index contains 286 entries
Processing "Genesis/1. Mose" - 3, file position: 12057 (0.17%)
Merging in-memory index to disk.
Index merger done in 0.001s
Cleared in-memory index, size before: 102457 and after: 665
Document had 245, took 0.132s @ 1852.52tps | in-memory index contains 88 entries
Processing "Numeri/4. Mose" - 1, file position: 16992 (0.24%)
Document had 394, took 0.219s @ 1802.54tps | in-memory index contains 200 entries
Processing "Exodus/2. Mose" - 2, file position: 26055 (0.37%)
Merging in-memory index to disk.
Index merger done in 0

And finally, we do the actual benchmark:

In [22]:
example_queries = [
    'Jesus', 'der Himmel', 'es stand geschrieben', 'Maria', 'herr', 'gott',
    'herr gott', 'moses', 'gott', 'jesus', 'engel', 'gabriel', 'johannes',
    'täufer', 'frucht', 'baum', 'meer', 'fisch', 'zimt', 'esel', 'abkomme'
]

def test(processor: QueryProcessor):
    global_start_time = time.time()
    for i, query in enumerate(example_queries):
        start_time = time.time()
        docs = []
        for result in processor.query(query):
            docs.append(result.as_reference())

        used_time = time.time() - start_time
        print(f'{i}) "{query}" took {used_time:.3f}s and found {len(docs)}')
        # logger.debug(f'{docs}')
    print(f'All queries took {(time.time() - global_start_time):.3f}s')


print('')
print('~~ Testing Index Queries (mode: NON_POSITIONAL) ~~')
test(BinaryQueryProcessor(index=bibel_index, mode=BinaryQueryProcessorMode.NON_POSITIONAL))
print('')
print('~~ Testing Index Queries (mode: TF-IDF) ~~')
test(BinaryQueryProcessor(index=bibel_index, mode=BinaryQueryProcessorMode.TFIDF_RANKING, simulate_sort=True))
print('')
print('~~ Testing Naïve Queries ~~')
test(FullScanQueryProcessor(documents_filename='bible.json', pipeline=PRE_PROCESSING))


~~ Testing Index Queries (mode: NON_POSITIONAL) ~~
Loading lookup table...
Loaded index with 15480 words in vocab and 0 documents with mode=BinaryQueryProcessorMode.NON_POSITIONAL.
0) "Jesus" took 0.036s and found 202
1) "der Himmel" took 0.064s and found 399
2) "es stand geschrieben" took 0.013s and found 56
3) "Maria" took 0.004s and found 18
4) "herr" took 0.149s and found 971
5) "gott" took 0.158s and found 1008
6) "herr gott" took 0.136s and found 761
7) "moses" took 0.044s and found 249
8) "gott" took 0.156s and found 1008
9) "jesus" took 0.037s and found 202
10) "engel" took 0.021s and found 124
11) "gabriel" took 0.001s and found 3
12) "johannes" took 0.013s and found 58
13) "täufer" took 0.002s and found 9
14) "frucht" took 0.013s and found 86
15) "baum" took 0.007s and found 42
16) "meer" took 0.036s and found 237
17) "fisch" took 0.008s and found 42
18) "zimt" took 0.001s and found 6
19) "esel" took 0.012s and found 66
20) "abkomme" took 0.000s and found 0
All queries took 